In [ ]:
# txt to bedgraph: we can change the window size 
# windows = 1bp + CPM
import pandas as pd
import os

# 1. 设置输入输出文件名
input_file = '675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_hmc_cxreport.txt'
output_file = '675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_hmc_cxreport_cpm_1bp.bedgraph'

print("步骤 1/3: 正在扫描全基因组以计算总信号量 (Normalization Factor)...")

# --- 第一步：计算全样本的总修饰计数 (Total Signal) ---
total_modified_counts = 0
chunks_for_sum = pd.read_csv(input_file, sep='\t', header=None, chunksize=1000000, usecols=[3])

for chunk in chunks_for_sum:
    total_modified_counts += chunk[3].sum()

print(f"检测到总修饰计数: {total_modified_counts}")

# 防止除以零
if total_modified_counts == 0:
    print("错误：总修饰计数为 0，请检查输入文件。")
    exit()

# --- 第二步：计算 CPM 并转换为 BedGraph ---
print("步骤 2/3: 正在计算 CPM 并写入临时文件...")
chunks = pd.read_csv(input_file, sep='\t', header=None, chunksize=1000000)

first_chunk = True
temp_file = 'temp_unsorted_cpm.bg'

for chunk in chunks:
    # 提取需要的列：0:chr, 1:pos, 3:modified_C
    df = pd.DataFrame()
    df['chrom'] = chunk[0]
    df['start'] = chunk[1] - 1  # 1bp window: 0-based start
    df['end'] = chunk[1]        # 1bp window: end
    
    # 【核心计算】：CPM = (当前位点计数 / 总计数) * 1,000,000
    df['value'] = (chunk[3] / total_modified_counts) * 1000000
    
    # 写入临时文件
    mode = 'w' if first_chunk else 'a'
    df.to_csv(temp_file, sep='\t', index=False, header=False, mode=mode)
    first_chunk = False

# --- 第三步：排序 ---
print("步骤 3/3: 正在进行全基因组坐标排序...")
# 使用系统命令 sort 确保 IGV 能够正常读取（IGV 对顺序要求极严）
os.system(f"sort -k1,1 -k2,2n {temp_file} > {output_file}")

# 删除临时文件
if os.path.exists(temp_file):
    os.remove(temp_file)

print(f"CPM 处理完成！生成文件：{output_file}")

In [ ]:
# bedgraph to bigwig 
import pyBigWig
import pandas as pd

def convert_to_bigwig(bedgraph_file, chrom_sizes_file, output_bw):
    # 1. 加载染色体大小并记录顺序
    print("正在加载染色体大小...")
    chrom_order = []
    chrom_sizes_dict = {}
    with open(chrom_sizes_file, 'r') as f:
        for line in f:
            if line.strip():
                parts = line.split()
                chrom_order.append(parts[0]) 
                chrom_sizes_dict[parts[0]] = int(parts[1])
    
    # 2. 创建 BigWig
    bw = pyBigWig.open(output_bw, "w")
    bw.addHeader([(chrom, chrom_sizes_dict[chrom]) for chrom in chrom_order])
    
    # 3. 读取数据
    print(f"正在读取 {bedgraph_file}...")
    df = pd.read_csv(bedgraph_file, sep='\t', header=None)
    df.columns = ['chrom', 'start', 'end', 'value']
    
    # 4. 按照 chrom.sizes 的顺序写入
    for chrom in chrom_order:
        group = df[df['chrom'] == chrom]
        if not group.empty:
            group = group.sort_values('start')
            print(f"正在写入 {chrom}...")
            bw.addEntries(
                group['chrom'].tolist(),
                group['start'].tolist(),
                ends=group['end'].tolist(),
                values=group['value'].tolist()
            )
    
    bw.close()
    print(f"转换成功！最终文件：{output_bw}")

# 填入你真实的文件名
my_input = '675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport.txt.50bp.CPM.bedgraph'
my_sizes = 'hg38.chrom.sizes'
my_output = '675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport.txt.50bp.CPM.bedgraph.bw'

# 执行
convert_to_bigwig(my_input, my_sizes, my_output)

In [ ]:
# The code for bigwig compare in a row
import os

# 1. 设置你的 BED 文件路径
bed_file = "FMR1_CGG_Annotation_fixed.bed"

# 2. 修改后的坐标提取逻辑
with open(bed_file, 'r') as f:
    line = f.readline().strip().split('\t')
    # 【核心修改点】：使用冒号分隔三个部分
    target_region = f"{line[0]}:{line[1]}:{line[2]}"

print(f"🎯 修正后的目标区域格式: {target_region}")

# 3. 文件列表
bw_hmc_list = [
    "675720-0427605774.targeted.GRCh38Decoy.markdup.CG.num_hmc_cxreport_cpm_1bp.bedgraph.bw",
    "675721-0427605775.targeted.GRCh38Decoy.markdup.CG.num_hmc_cxreport_cpm_1bp.bedgraph.bw",
    "675722-0427605773.targeted.GRCh38Decoy.markdup.CG.num_hmc_cxreport_cpm_1bp.bedgraph.bw",
    "675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_hmc_cxreport_cpm_1bp.bedgraph.bw"
]

bw_modc_list = [
    "675720-0427605774.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport_cpm_1bp.bedgraph.bw",
    "675721-0427605775.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport_cpm_1bp.bedgraph.bw",
    "675722-0427605773.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport_cpm_1bp.bedgraph.bw",
    "675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_modc_cxreport_cpm_1bp.bedgraph.bw"
]

# 4. 循环处理
for hmc, modc in zip(bw_hmc_list, bw_modc_list):
    sample_id = hmc.split('-')[0]
    output_ratio_bw = f"{sample_id}_FMR1_CGG_Ratio_Track.bw"
    
    if os.path.exists(hmc) and os.path.exists(modc):
        print(f"\n▶️ 正在提取区域并计算比值: {sample_id}...")
        
        # 运行命令
        !bigwigCompare -b1 {hmc} -b2 {modc} \
                      --operation ratio \
                      --region "{target_region}" \
                      --pseudocount 1 \
                      --binSize 1 \
                      -o {output_ratio_bw}
        
        if os.path.exists(output_ratio_bw):
            print(f"✅ 已生成局部比值文件: {output_ratio_bw}")
            
print("\n--- 处理结束 ---")

In [ ]:
import deeptools 

# 定义你的文件名，方便后续修改
bw_files = "675720-0427605774.targeted.GRCh38Decoy.markdup.CG.num_mc_cxreport.txt.50bp.bedgraph.bw 675721-0427605775.targeted.GRCh38Decoy.markdup.CG.num_mc_cxreport.txt.50bp.bedgraph.bw 675722-0427605773.targeted.GRCh38Decoy.markdup.CG.num_mc_cxreport.txt.50bp.bedgraph.bw 675723-0427605772.targeted.GRCh38Decoy.markdup.CG.num_mc_cxreport.txt.50bp.bedgraph.bw"
labels = "gNHG3 Interal_control gCGG gScr"
output_npz = "results_summary.npz"

# 运行 multiBigwigSummary
# --binSize 10000: 全基因组分析常用 10kb，如果是针对 FMR1 局部可以设小一点（如 500）
!multiBigwigSummary bins \
    --bwfiles {bw_files} \
    --labels {labels} \
    --outFileName {output_npz} \
    --binSize 10000 \
    -p max